# Import libraries

In [1]:
import os
import pandas as pd
import tweepy
import json
import urllib.request
import random 
import numpy as np
import datetime
from datetime import date
import warnings
import requests

# load json with country names and codes
with urllib.request.urlopen("https://raw.githubusercontent.com/dquintani/GreenhouseData/master/supplemental/STANDARD_COUNTRY_DICT_ISO3.json") as url:
    STANDARD_COUNTRY_DICT = json.loads(url.read().decode())
    
pd.set_option('display.max_colwidth', None)

In [2]:
# environmental secrets when working locally
from dotenv import load_dotenv
try:
    load_dotenv()
    load_dotenv("../../../")
    print("envs loaded")
except:
    print("envs failed to load")
    pass

envs loaded


# Set up tweepy

In [3]:
#USING GITHUB ENVS

#test
GHD_TWITTER_APIKEY= os.environ["GHD_TWITTER_APIKEY"]
GHD_TWITTER_APIKEYSECRET = os.environ["GHD_TWITTER_APIKEYSECRET"]

GHD_TWITTER_BEARERTOKEN = os.environ["GHD_TWITTER_BEARERTOKEN"]
GHD_TWITTER_ACCESSTOKEN = os.environ["GHD_TWITTER_ACCESSTOKEN"]
GHD_TWITTER_ACCESSTOKENSECRET = os.environ["GHD_TWITTER_ACCESSTOKENSECRET"]
GHD_TWITTER_CLIENTID = os.environ["GHD_TWITTER_CLIENTID"]
GHD_TWITTER_CLIENTSECRET = os.environ["GHD_TWITTER_CLIENTSECRET"]

# Authenticate to Twitter
auth = tweepy.OAuthHandler(GHD_TWITTER_APIKEY, GHD_TWITTER_APIKEYSECRET)
auth.set_access_token(GHD_TWITTER_ACCESSTOKEN, GHD_TWITTER_ACCESSTOKENSECRET)

# Create API object 
api = tweepy.API(auth)
api

In [4]:
#api.rate_limit_status()

# Retrieve log

In [5]:
id_log = pd.read_csv("id_log.csv", index_col = 0)
id_log.reply_datetime = pd.to_datetime(id_log.reply_datetime)
id_log

,reply_datetime,username
tweet_id,,
1510165308602040320,2022-01-10 00:00:00.000000,UNEP
1509886513923989507,2022-03-12 00:00:00.000000,USDOT
1510559067311206401,2022-04-01 00:00:00.000000,quicoferrer
1509179737972912132,2022-04-05 00:35:02.010471,manuvimalassery
1508730985550127114,2022-04-05 00:36:33.250239,IBCSOLAR_Int
...,...,...
1552670252013682689,2022-08-02 23:10:59.680888,KoreaEconInst
1554452589026189315,2022-08-03 08:15:15.605638,CH4Observatory
1554285889404039168,2022-08-03 08:15:29.465037,OmairTAhmad


# Functions

In [6]:
def REPLY_MESSAGE_GENERATOR(code, country_place = True):
    
    # code can be a country or another place (such as global)
    if country_place == True:
        place = STANDARD_COUNTRY_DICT[code]
        link = f"https://dquintani.github.io/GreenhouseData/country_data/{code}_{STANDARD_COUNTRY_DICT[code]}/"
    elif country_place == False:
        place = code
        link = "https://dquintani.github.io/GreenhouseData/global"
        
    
    text1  = [
        f"{place} greenhouse gas emission data and figures 👇\n\n"
    ]
    
    text2 = [""
             # "Hi there! Thank you for tweeting about greenhouse gas emissions.\n\n",
             # "Hello there! Thank you for tweeting about greenhouse gas emissions.\n\n",
             # "Hi there! Thank you for tweeting about GHG emissions.\n\n",
             # "Hello there! Thank you for tweeting about GHG emissions.\n\n",
            ]
    
    
    text3 = ["",
             "",
            ]
    
    text4 = [""
            # "I may hold information you are interested in: ",
            #  "Perhaps I might have something of your interest: ",
            #  "I have something that might be useful to you: "
            ]
    
    if country_place == True:
        text5 = [""
            # f"GHG emission datasets and figures for {place}! ",
            # f"GHG emission figures and lots of datasets for {place}! "
        ]
        
    elif country_place == False:
        text5 = ["Lots of GHG emission data and figures on a global and international scale!"]
        
    text6 = [
             "If you like this kind of stuff be sure to check out my website for many more datasets and figures! Thank you and have a wonderful day 🤖",
             "If interested be sure to check out my website for many more datasets and figures! Thank you and take care! 🤖",
            ]
    
    

    FINAL_MESSAGE = random.choice(text1) + random.choice(text2) + random.choice(text3) + random.choice(text4) + random.choice(text5) + random.choice(text6)# + link.replace(" ","%20")
    
    return FINAL_MESSAGE


###### ------------------ EXAMPLE --------------------------------------

MESSAGE =  REPLY_MESSAGE_GENERATOR("IRN")
print(MESSAGE)

Iran, Islamic Republic of greenhouse gas emission data and figures 👇

If you like this kind of stuff be sure to check out my website for many more datasets and figures! Thank you and have a wonderful day 🤖


In [7]:
def TWEET_FINDER(message, search_count = 20, result_type = "mixed"):

    print(f" > Searching for {search_count} tweets containing \"{message}\"")
    
    #FIND TWEETS
    tweet_list = api.search_tweets(message, 
                                   result_type=result_type, 
                                   count=search_count)
    
    print(f" > found {len(tweet_list)} tweets")
    
    # MAKE DF OF TWEETS
    tweet_df = pd.DataFrame(columns=["username",
                                      "followers",
                                      "date",
                                      "time",
                                      "location",
                                      "verified",
                                      # "link_and_id",
                                      "id",
                                      "message",
                                     ])
    for i, j in enumerate(tweet_list):
        tweet_df.loc[i] = [j.user.screen_name, 
                            j.user.followers_count, 
                            j.created_at.date(), 
                            j.created_at.time(), 
                            j.user.location, 
                            j.user.verified, 
                            # f"https://twitter.com/twitter/statuses/{j.id}",
                            j.id,
                            api.get_status(j.id, tweet_mode="extended").full_text,
                           ]
        

    #SORT TWEETS BY TIME
    tweet_df.sort_values(["date","time"], ascending=False)
    
    return tweet_df

# ------------------------------------------------------------------------------------------------------------

# tweet_df = TWEET_FINDER("global greenhouse gas emissions", result_type="mixed")
# tweet_df.head()

In [8]:
def TWEET_FILTER(invalidity_period_days = 10, remove_retweets = True):
    
    ###############################
    #      FILTER 1: name log      #
    ################################
    
    # RETRIEVE ID AND NAME LOG
    name_log = id_log.copy()
    # name_log["username"] = np.nan
    # for i in range(len(name_log)):
    #     try:
    #         name_log.loc[name_log.index[i],"username"] = api.get_status(name_log.index[i]).user.screen_name
    #     except:
    #         pass # this was added because some users were suspended and thus the code failed when trying to fetch their status
        
    # Current time and invalidity date
    now = datetime.datetime.now()
    print(f" > days of invalidity: {invalidity_period_days}")
    invalidity_date = now - datetime.timedelta(days=invalidity_period_days)
    
    #create list of invalid usernames
    name_log_invalid_list = list(name_log[name_log.reply_datetime > invalidity_date].username)
    print(f" > Number of invalids: {len(name_log_invalid_list)}")
    
    # Final filter
    tweet_df_filtered1 = tweet_df[-tweet_df.username.isin(name_log_invalid_list)]
    filter_count = len(tweet_df) - len(tweet_df_filtered1)
    print(f" > {filter_count} usernames filtered out")
    
    
    
    
    
    ###################################
    #    FILTER 2: retweeted tweets   #
    ##################################$
    
    warnings.filterwarnings("ignore")

    #create new df and add "is_a_retweet" columns
    tweet_df_filtered2 = tweet_df_filtered1
    tweet_df_filtered2["is_a_retweet"] = np.nan
        
    for i, j in enumerate(tweet_df_filtered1["id"]):
        
        tweet = api.get_status(j)
        
        try:
            if type(tweet.retweeted_status) == tweepy.models.Status:
                tweet_df_filtered2.loc[:,"is_a_retweet"][i] = True
        except:
            tweet_df_filtered2.loc[:,"is_a_retweet"][i] = False
            
    warnings.filterwarnings("default")
    
    # Filter out tweets that are actually just retweets
    if remove_retweets == True:
        starting = len(tweet_df_filtered2)
        tweet_df_filtered2 = tweet_df_filtered2[tweet_df_filtered2.is_a_retweet==False]
        retweets_removed = starting - len(tweet_df_filtered2)
        print(f" > retweets removed: {retweets_removed}")

    
    return tweet_df_filtered2
    
# ----------------------------------------------------------------------------------------------------------------    

# tweet_df_filtered = TWEET_FILTER()
# tweet_df_filtered

In [9]:
def TWEET_SELECTOR(count, priority, remove_col_value = ["column", "value"], max_followers = None):
    
    SELECTED_TWEETS = []
    df = tweet_df_filtered
    
    #priority
    if priority == "follower_count":
        df = df.sort_values("followers", ascending=False)
    if priority == "verified":
        df = df.sort_values("verified", ascending=False)
    
    # remove specific value in a columns?
    if remove_col_value != ["column", "value"]:
        df = df[df[remove_col_value[0]] != remove_col_value[1]]
        print(f"removed [column, value] = {remove_col_value}")
    
    #max followers?
    if max_followers != None:
        df = df[df.followers < max_followers]
        print(f"max followers: {max_followers}")
    
    SELECTED_TWEETS = list(df["id"][:count])
    return SELECTED_TWEETS

# -----------------------------------------------------------------------

# SELECTED_TWEETS = TWEET_SELECTOR(count = 3, 
#                priority = "verified", 
#                remove_col_value = ["verified", False],
#                max_followers = 1000000)

In [10]:
def TWEET_IMAGE_SAVER_AND_SELECTOR(code, max_count = 4, is_global=False):
    
    # Setup requests to find images in repository
    user = "dquintani"
    repo = "GreenhouseData"
    url = f"https://api.github.com/repos/{user}/{repo}/git/trees/master?recursive=1"
    r = requests.get(url)
    res = r.json()
    list_dir = []
    for file in res["tree"]:
        list_dir.append(file["path"])    
    
    # save randomly
    if is_global==True:
        path_dir = "global/"
        list_figs = [i for i in list_dir if i.startswith(path_dir)]
        try:
            selected_figs_paths = random.sample(list_figs, max_count)
        except:
            selected_figs_paths = []
    
    elif is_global == False: #ie is a country code
        path_dir = f"country_data/{code}_{STANDARD_COUNTRY_DICT[code]}/figures/"
        list_figs = [i for i in list_dir if i.startswith(path_dir)]
        
        selected_figs_paths = []
        # selected_figs_paths = random.sample(list_figs, max_count-1)
        selected_figs_paths.append(path_dir + f"{code}_relative_totals.png")
        selected_figs_paths.append(path_dir + f"{code}_GCP_Country_Highlight.png")
        selected_figs_paths.append(path_dir + f"{code}_GCP_1.png")
        selected_figs_paths.append(path_dir + f"{code}_Minx_top20_subsectors.png")
        display(selected_figs_paths)
        

    #DOWNLOAD AND SAVE THEM
    for i,j in enumerate(selected_figs_paths):
        image_url = f"https://dquintani.github.io/GreenhouseData/{j}"
        img_data = requests.get(image_url).content
        with open(f'image{i+1}.jpg', 'wb') as handler:
            handler.write(img_data)
        
    
    media_ids = []
    for i in range(max_count):
        try:
            res = api.media_upload(f"image{i+1}.jpg")
            media_ids.append(res.media_id)
        except:
            pass
    print(f" > saved and selected {len(media_ids)} images")
    return media_ids
    
# ---------------------------------------------------------------

# TWEET_IMAGE_SAVER_AND_SELECTOR("BES", 4)

In [11]:
def TWEET_REPLIER_AND_LOGGER(SELECTED_TWEETS, MESSAGE, media_ids):
    
    if len(SELECTED_TWEETS)>0:
        
        for i,j in enumerate(SELECTED_TWEETS):

            # SEND TWEET
            if len(media_ids)==0:
                medi_ids = None
                
            try:
                api.update_status(status = MESSAGE, 
                                  in_reply_to_status_id = j,
                                  auto_populate_reply_metadata=True,
                                  media_ids=media_ids)

                # LOG TWEET ID (AND THUS USERNAME) and datetime
                id_log.loc[j] = [datetime.datetime.now(), api.get_status(j).user.screen_name]

                SUCCESS = True
                print(" > TWEET SUCCESSFUL!!")
                
            except:
                SUCCESS = False
                print("\n > OH NO! Something sketchy is going on with that username! Nothing tweeted. False.")
        

    if len(SELECTED_TWEETS) == 0:
        print("\n > OH NO! NO TWEETS WERE FOUND NOR TWEETED")
        SUCCESS = False
    # save log
    id_log.to_csv("id_log.csv")
    
    return SUCCESS
    
# -------------------------------------------------



In [12]:
id_log

,reply_datetime,username
tweet_id,,
1510165308602040320,2022-01-10 00:00:00.000000,UNEP
1509886513923989507,2022-03-12 00:00:00.000000,USDOT
1510559067311206401,2022-04-01 00:00:00.000000,quicoferrer
1509179737972912132,2022-04-05 00:35:02.010471,manuvimalassery
1508730985550127114,2022-04-05 00:36:33.250239,IBCSOLAR_Int
...,...,...
1548855621755277314,2022-07-19 23:02:08.137098,TheFastMode
1546462193687834629,2022-07-20 08:08:36.421388,junkfoodwisdom
1548274777684463619,2022-07-20 08:08:49.773927,NesEvan9


# EXE: RANDOM COUNTRY

In [13]:
TWEETS_TO_SEND_COUNT = 2

SUCCESS_LIST = []
while SUCCESS_LIST.count(True) < TWEETS_TO_SEND_COUNT:
    
    print("Starting success list", SUCCESS_LIST)

    # 0. Generate message (required, select False for global):
    code = random.choice(list(STANDARD_COUNTRY_DICT.keys()))
    country_place = True

    # 1. Get the DF:
    message = f"greenhouse gas emissions {STANDARD_COUNTRY_DICT[code]}" #default: ""
    search_count = 20 # default: 20
    result_type = "mixed" #no default, options: mixed, popular, recent

    # 2. Filter: 
    invalidity_period_days = 20 # default 20
    remove_retweets = True #default: True


    # 3. Select the target tweets by id and count:
    count = 1 # no default
    priority = "follower_count" # no defualt, options: "follower_count", "verified" 
    remove_col_value = ["column", "value"] # default: ["column", "value"]
    max_followers = None # default: None

    # 4. Select and save random figures:
    max_count = 4 #default 4
    is_global = False # default False


    ######################################
    #             EXECUTE                #
    ######################################

    MESSAGE = REPLY_MESSAGE_GENERATOR(code, country_place = country_place)
    tweet_df = TWEET_FINDER(message, search_count = search_count, result_type = result_type)
    
    if len(tweet_df)==0:
        print(" > empty df: NEXT ITERATION\n")
        continue
        
    tweet_df_filtered = TWEET_FILTER(invalidity_period_days = invalidity_period_days, remove_retweets = remove_retweets)
    SELECTED_TWEETS = TWEET_SELECTOR(count = count, priority = priority, 
                                     remove_col_value = remove_col_value, 
                                     max_followers = max_followers)
    media_ids = TWEET_IMAGE_SAVER_AND_SELECTOR(code, max_count = max_count, is_global=is_global)


    SUCCESS_LIST.append(TWEET_REPLIER_AND_LOGGER(SELECTED_TWEETS, MESSAGE, media_ids))
    print(f"success list: {SUCCESS_LIST} \n\n\n")

print("\n\n FINISHED WHILE LOOP")

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Saint Martin (French part)"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Kuwait"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Saint Lucia"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Christmas Island"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Portugal"
 > found 2 tweets
 > days of invalidity: 20
 > Number of invalids: 134
 > 0 usernames filtered out
 > retweets removed: 0


['country_data/PRT_Portugal/figures/PRT_relative_totals.png',
 'country_data/PRT_Portugal/figures/PRT_GCP_Country_Highlight.png',
 'country_data/PRT_Portugal/figures/PRT_GCP_1.png',
 'country_data/PRT_Portugal/figures/PRT_Minx_top20_subsectors.png']

 > saved and selected 4 images
 > TWEET SUCCESSFUL!!
success list: [True] 



Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions Sri Lanka"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions Senegal"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions El Salvador"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions Saint Barthélemy"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions South Georgia and the South Sandwich Islands"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions San Marino"
 > found 0 tweets
 >

['country_data/QAT_Qatar/figures/QAT_relative_totals.png',
 'country_data/QAT_Qatar/figures/QAT_GCP_Country_Highlight.png',
 'country_data/QAT_Qatar/figures/QAT_GCP_1.png',
 'country_data/QAT_Qatar/figures/QAT_Minx_top20_subsectors.png']

 > saved and selected 4 images
 > TWEET SUCCESSFUL!!
success list: [True, True] 





 FINISHED WHILE LOOP


# EXE: TIER COUNTRIES

In [14]:
#tier 1
TIER_1_COUNTRIES = ["USA", "GBR", "DEU", "CHN", "IND","AUS", "CHE"]
TIER_2_COUNTRIES = ["ITA", "IRL", "CAN", "JPN", "BRA", "IDN", "RUS","ZAF", "NZL", "FRA", "IRN", "MEX", "TUR", "POL"]
TIER_1_AND_2 = TIER_1_COUNTRIES + TIER_2_COUNTRIES
# TIER_1_AND_2

In [15]:
TWEETS_TO_SEND_COUNT = 0

SUCCESS_LIST = []
while SUCCESS_LIST.count(True) < TWEETS_TO_SEND_COUNT:
    
    print("Starting success list", SUCCESS_LIST)

    # 0. Generate message (required, select False for global):
    code = random.choice(list(TIER_1_AND_2))
    country_place = True

    # 1. Get the DF:
    message = f"greenhouse gas emissions {STANDARD_COUNTRY_DICT[code]}" #default: ""
    search_count = 20 # default: 30
    result_type = "mixed" #no default, options: mixed, popular, recent

    # 2. Filter: 
    invalidity_period_days = 20 # default 20
    remove_retweets = True #default: True


    # 3. Select the target tweets by id and count:
    count = 1 # no default
    priority = "follower_count" # no defualt, options: "follower_count", "verified" 
    remove_col_value = ["column", "value"] # default: ["column", "value"]
    max_followers = None # default: None

    # 4. Select and save random figures:
    max_count = 4 #default 4
    is_global = False # default False


    ######################################
    #             EXECUTE                #
    ######################################

    MESSAGE = REPLY_MESSAGE_GENERATOR(code, country_place = country_place)
    tweet_df = TWEET_FINDER(message, search_count = search_count, result_type = result_type)
    tweet_df_filtered = TWEET_FILTER(invalidity_period_days = invalidity_period_days, remove_retweets = remove_retweets)
    
    if len(tweet_df)==0:
        print(" > empty df: NEXT ITERATION\n")
        continue
        
    SELECTED_TWEETS = TWEET_SELECTOR(count = count, priority = priority, 
                                     remove_col_value = remove_col_value, 
                                     max_followers = max_followers)
    media_ids = TWEET_IMAGE_SAVER_AND_SELECTOR(code, max_count = max_count, is_global=is_global)


    SUCCESS_LIST.append(TWEET_REPLIER_AND_LOGGER(SELECTED_TWEETS, MESSAGE, media_ids))
    print(f"success list: {SUCCESS_LIST} \n\n\n")
    
print("\n\n FINISHED WHILE LOOP")



 FINISHED WHILE LOOP


# check log

In [16]:
# log = pd.read_csv("id_log.csv", index_col=0)
# log["username"] = np.nan
# for i in range(len(log)):
#         try:
#             log.loc[log.index[i],"username"] = api.get_status(log.index[i]).user.screen_name
#         except:
#             pass # this was added because some users were suspended and thus the code failed when trying to fetch their status

In [17]:
# log.to_csv("id_log.csv")

# Tweet profile of the day

In [18]:
user = "greenhousedata"
tl = api.user_timeline(screen_name = user, count=30, exclude_replies=True, include_rts=False)
latest_tweet = tl[0]
latest_tweet_hrs_ago = latest_tweet.created_at
latest_tweet_minus_24hr = latest_tweet_hrs_ago  + datetime.timedelta(hours=17)
now = datetime.datetime.now(tz=datetime.timezone.utc)


if latest_tweet_minus_24hr < now:
    code = random.choice(list(STANDARD_COUNTRY_DICT.keys()))
    print(STANDARD_COUNTRY_DICT[code])
    media_ids = TWEET_IMAGE_SAVER_AND_SELECTOR(code, 4)
    MESSAGE = f"Greenhouse Data's country profile of the day: {STANDARD_COUNTRY_DICT[code]}\n\nMore greenhouse gas emission data and figures of this country: https://dquintani.github.io/GreenhouseData/country_data/{code}_{STANDARD_COUNTRY_DICT[code]}/"

    api.update_status(status = MESSAGE, 
                      media_ids=media_ids)
    print("country profile tweet SUCCESS!")
    
else:
    print("TOOO SOON!!")

TOOO SOON!!
